In [7]:
import os
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import load_model

def test_model(model, test_data):
    # Initialize metrics
    precision_metric = tf.keras.metrics.Precision()
    recall_metric = tf.keras.metrics.Recall()
    accuracy_metric = tf.keras.metrics.BinaryAccuracy()
    
    # Iterate over the test data
    for x, y_true in test_data:
        y_pred = model.predict(x)
        
        # Update metrics
        precision_metric.update_state(y_true, y_pred)
        recall_metric.update_state(y_true, y_pred)
        accuracy_metric.update_state(y_true, y_pred)
    
    # Calculate final metric values
    precision = precision_metric.result().numpy()
    recall = recall_metric.result().numpy()
    accuracy = accuracy_metric.result().numpy()
    
    # Calculate F1 Score
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-7)
    
    # Return metrics as a dictionary
    return {
        'Precision': precision,
        'Recall': recall,
        'Binary_Accuracy': accuracy,
        'F1_Score': f1_score
    }

def main():
    # Specify the root folder containing model subfolders
    models_root = r"C:\Users\acer\Desktop\result_padding_exp\result_0px\models"
    
    # Specify the CSV file path for results
    results_file = r"C:\Users\acer\Desktop\result_padding_exp\result_0px\basemodel_test_result.csv.txt"
    
    # Specify the directory containing your test data
    data_dir = r"C:\Users\acer\Desktop\test_data\TF_TMJOA_jpg_x_0px_test"
    
    # Load your test data
    print("Loading test data...")
    test_data = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        image_size=(224, 224),
        batch_size=32,
        shuffle=False
    )
    
    # Create a list to store results
    results_list = []
    
    # Iterate through model folders
    for model_folder in os.listdir(models_root):
        model_folder_path = os.path.join(models_root, model_folder)
        
        # Skip if not a directory
        if not os.path.isdir(model_folder_path):
            continue
            
        print(f"\nProcessing models in folder: {model_folder}")
        
        # Iterate through all .h5 files in the model folder
        for model_file in os.listdir(model_folder_path):
            if model_file.endswith(".h5"):
                model_path = os.path.join(model_folder_path, model_file)
                full_model_name = f"{model_folder}/{model_file}"
                
                print(f"Testing model: {full_model_name}")
                
                try:
                    # Load the model
                    model = tf.keras.models.load_model(model_path)
                    
                    # Test the model and get metrics
                    metrics = test_model(model, test_data)
                    
                    # Add model name and metrics to results
                    result_row = {'Model_Name': full_model_name}
                    result_row.update(metrics)
                    results_list.append(result_row)
                    
                    print(f"Successfully tested {full_model_name}")
                    
                except Exception as e:
                    print(f"Error testing {full_model_name}: {str(e)}")
                    # Add failed model to results with NaN values
                    result_row = {
                        'Model_Name': full_model_name,
                        'Precision': float('nan'),
                        'Recall': float('nan'),
                        'Binary_Accuracy': float('nan'),
                        'F1_Score': float('nan')
                    }
                    results_list.append(result_row)
    
    # Create DataFrame from results
    results_df = pd.DataFrame(results_list)
    
    # Reorder columns to put Model_Name first
    columns_order = ['Model_Name', 'Precision', 'Recall', 'Binary_Accuracy', 'F1_Score']
    results_df = results_df[columns_order]
    
    # Round numeric values to 4 decimal places
    numeric_columns = ['Precision', 'Recall', 'Binary_Accuracy', 'F1_Score']
    results_df[numeric_columns] = results_df[numeric_columns].round(4)
    
    # Save results to CSV
    results_df.to_csv(results_file, index=False)
    print(f"\nResults saved to {results_file}")
    
    # Display results
    print("\nTest Results Summary:")
    print(results_df.to_string())
    
    # Display summary statistics grouped by model folder
    print("\nSummary Statistics by Model Type:")
    summary_by_model = results_df.copy()
    summary_by_model['Model_Type'] = summary_by_model['Model_Name'].apply(lambda x: x.split('/')[0])
    summary_stats = summary_by_model.groupby('Model_Type')[numeric_columns].mean().round(4)
    print(summary_stats.to_string())

if __name__ == "__main__":
    main()

Loading test data...
Found 6550 files belonging to 2 classes.

Processing models in folder: DenseNet201
Testing model: DenseNet201/DenseNet201_bo40_lr0001.h5
check1 C:\Users\acer\Desktop\result_padding_exp\result_0px\models\DenseNet201\DenseNet201_bo40_lr0001.h5
Error testing DenseNet201/DenseNet201_bo40_lr0001.h5: Error when deserializing class 'Conv2D' using config={'name': 'conv1/conv', 'trainable': False, 'dtype': 'float32', 'filters': 64, 'kernel_size': [7, 7], 'strides': [2, 2], 'padding': 'valid', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'groups': 1, 'activation': 'linear', 'use_bias': False, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}.

Exception encountered: Argument `name` must be a string and cannot contain character `/`. Received:

KeyboardInterrupt: 